<a href="https://colab.research.google.com/github/deniskapel/GenerativeChitChat/blob/master/Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
mkdir data
wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1uSgX8EaXtSR1yZgs-pJGYiZXi7tPEBrE' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1uSgX8EaXtSR1yZgs-pJGYiZXi7tPEBrE" -O data/qa_data.jsonl.zip && rm -rf /tmp/cookies.txt
unzip data/qa_data.jsonl.zip -d data

In [ ]:
%%bash
wget "https://raw.githubusercontent.com/deniskapel/GenerativeChitChat/master/requirements.txt"
pip install -r requirements.txt

In [9]:
import codecs
import json
import math
import numpy as np
import pandas as pd

import random
import json
import torch
from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm

from matplotlib import pyplot as plt

import youtokentome as yttm

In [10]:
!sed 5q data/qa_data.jsonl

{"question": "долго ли идут деньги с яндексденег на карту visa?", "category": "Бизнес, Финансы", "responses": ["нет. прорыв 35 ;)"]}
{"question": "можно ли зарегистрировать авто в другом регионе", "category": "Авто, Мото", "responses": ["можно на родственника из того региона.. .  а потом ездить по доверке"]}
{"question": "что делать если у меня очень тонкие ногти а хочется их отрастить?", "category": "Красота и Здоровье", "responses": ["витамины и умная эмаль (каждый день)", "ванночки с морской солью. с вечера мажь ногти сверху йодом. не бойся, до утра все впитается.", "умная эмаль, витамины, йод, и поменьше крась лаком ", "лаки фирмы trind производство usa + кальций"]}
{"question": "в чем отличие медитации от йоги?", "category": "Спорт", "responses": ["букв в йоге меньше", "в медитации ты просто сидишь и мммммычишь. а в йоге всяко разные упражнения вытворяешь", "в медитации вроде просто тупо сидишь и успокаеваешься, а в йоге еще и ноги за уши закидывать надо"]}
{"question": "когда нач

In [11]:
with codecs.open("data/qa_data.jsonl", encoding='utf-8-sig') as reader:
    lines = reader.read().split("\n")
    lines = list(map(json.loads, filter(None, lines)))

data = []
for line in tqdm(lines):
    for response in line['responses']:
        data.append(
            {'question': line['question'].lower(),
             'category': line['category'],
             'response': response.lower()})

del lines
df = pd.json_normalize(data)
del data

  0%|          | 0/2808811 [00:00<?, ?it/s]

In [12]:
with open('data/for_bpe.txt', 'w', encoding='utf-8') as f:
    for q, r in zip(df.question, df.response):
        f.write(q + "\t" + r + '\n')
        
# bpe params
vocab_size = 30_000
model_path = 'data/my_pretrained_bpe_lm.model'

In [13]:
%%time
# обучаем
# раскомментируйте этот код, чтобы обучить bpe
yttm.BPE.train(data='data/for_bpe.txt', vocab_size=vocab_size, model=model_path)

CPU times: user 1min 47s, sys: 6.72 s, total: 1min 53s
Wall time: 38.9 s


In [14]:
tokenizer = yttm.BPE(model=model_path)

In [21]:
tokenizer.vocab()[:10]

['<PAD>', '<UNK>']

In [23]:
tokenizer.encode(df.question.tolist()[0], bos=True, eos=True, output_type=yttm.OutputType.SUBWORD)

['<BOS>',
 '▁долго',
 '▁ли',
 '▁идут',
 '▁деньги',
 '▁с',
 '▁яндекс',
 'дене',
 'г',
 '▁на',
 '▁карту',
 '▁vis',
 'a?',
 '<EOS>']